In [9]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
import gmaps
from urllib.request import urlopen
import json
import requests
import difflib
import sys
import datetime

# Google developer API key
sys.path.append('../')
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [11]:
data_df = pd.read_csv(os.path.join('./DCC_Data/', 'FinalDatasetAirplaneCrashes'))

In [12]:
df = data_df[(data_df["Fatalities"] >= 0 ) & (abs(data_df["Latitude"]) >= 0 )]
df = df.replace(np.nan, '', regex=True)

In [13]:
df['Aboard'] = pd.to_numeric(df['Aboard'])
df['Fatalities'] = pd.to_numeric(df['Fatalities'])

In [14]:
df['Percent Survived'] = df.apply(lambda x: 100 if x['Fatalities'] ==0 else x['Fatalities']/x['Aboard']*100, axis =1)
df_test =df.iloc[0:5]

In [15]:
def nearest(row):#https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city
    lat=row['Latitude']
    lon=row['Longitude']
    params = {
        "key": gkey,
        "radius": "50000",
        "location" : str(row["Latitude"]) + "," + str(row["Longitude"])
         }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    #pdb.set_trace()  
    try:
        country = response['results'][0]['vicinity'] 
    except:
        country = ''
    #print(requests.get(base_url, params=params).url)
    #print(json.dumps(response, indent=4, sort_keys=True))
    return(country) 

In [16]:
def getplace(row):#https://stackoverflow.com/questions/20169467/how-to-convert-from-longitude-and-latitude-to-country-or-city
    lat=row['Latitude']
    lon=row['Longitude']
    params = {
        "key": gkey,
        "latlng" : str(row["Latitude"]) + "," + str(row["Longitude"])
         }
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    response = requests.get(base_url, params=params).json()  
    
    try:
        components = response['results'][0]['address_components']
        country = town = None
        for c in components:
            if "country" in c['types']:
                country = c['long_name']
    except:
        #country = nearest(row)
        country = ''
    return(country)

In [ ]:
df['Country'] = df.apply(getplace, axis=1)
df['Nearest_Vicinity'] = df.apply(nearest, axis=1)
df.to_csv("df_with_country_and_nearest.csv")
df.head()

In [ ]:
#date = list(df['Date'].values)
#operator = list(df["Operator"].values)

text_list=[]
#Also instead of \n you must use html's <br/>
for index, row in df.iterrows():
    text_list.append('Information: <br/>' + 
                     "Date: " + str(row['Date']) + "<br/>" + 
                     "Time: " + str(row['Time']) + "<br/>" +
                     "Operator: " + str(row['Operator']) + "<br/>" +
                     "Flight #: " + str(row['Flight #']) + "<br/>" +
                     "Route: " + str(row['Route']) + "<br/>" +
                     "AC Type: " + str(row['AC Type']) + "<br/>" +
                     "Registration: " + str(row['Registration']) + "<br/>" +
                     "cn/ln: " + str(row['cn/ln']) + "<br/>" +
                     "Aboard: " + str(row['Aboard']) + "<br/>" +
                     "Aboard Passengers: " + str(row['Aboard Passengers']) + "<br/>" +
                     "Aboard Crew: " + str(row['Aboard Crew']) + "<br/>" +
                     "Fatalities: " + str(row['Fatalities']) + "<br/>" +
                     "Fatalities Passengers: " + str(row['Fatalities Passengers']) + "<br/>" +
                     "Fatalities Crew: " + str(row['Fatalities Crew']) + "<br/>" +
                     "Ground: " + str(row['Ground']) + "<br/>" +
                     "Summary: " + str(row['Summary']) + "<br/>" +
                     "Location: " + str(row['Location']) + "<br/>" +
                     "Latitude: " + str(row['Latitude']) + "<br/>" +
                     "Latitude: " + str(row['Latitude']) + "<br/>" +
                     "geometry: " + str(row['geometry']) + "<br/>"
                    )


In [ ]:
rating = df["Fatalities"]
locations = df[["Latitude", "Longitude"]]

# Plot Heatmap
fig = gmaps.figure()
figure_layout = {
    'width': '2000px',
    'height': '2000px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations.dropna()[0:1000], 
                                 weights=rating.dropna()[0:1000], 
                                 dissipating=True, 
                                 max_intensity=df['Fatalities'].max()/2,
                                 point_radius=15,
                                 opacity=0.95)
# Add layer
fig.add_layer(heat_layer)
#markers = gmaps.marker_layer(locations.dropna()[0:5])
#info_box_template = ""
#plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]
marker_layer = gmaps.marker_layer(locations.dropna()[0:50],
                                  info_box_content=text_list[0:50])
# Display figure
fig.add_layer(marker_layer)
fig

In [17]:
planecrash_df = pd.read_csv("./D")

FileNotFoundError: [Errno 2] No such file or directory: './df_with_country_with nearest.csv'

In [371]:
gdp_df = pd.read_csv("./DCC_Data/gdp_data.csv")
#gdp_df['Year'] = gdp_df['Year'].astype('int64')
gdp_df.astype({'Year':'int64'}).dtypes

Country            object
Code               object
Year                int64
GDP-Growth        float64
GDP-Per-Capita    float64
GDP               float64
dtype: object

In [372]:
#gdp_df['Year'] = gdp_df['Year'].astype(str)
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15048 entries, 0 to 15047
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         15048 non-null  object 
 1   Code            15048 non-null  object 
 2   Year            15048 non-null  int64  
 3   GDP-Growth      11539 non-null  float64
 4   GDP-Per-Capita  11391 non-null  float64
 5   GDP             11394 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 705.5+ KB


In [373]:
#planecrash_df['Year'] = planecrash_df.apply(lambda x: float(x['Date'][0:4])     ,axis=1)
planecrash_df['Year'] = planecrash_df.apply(lambda x: float(x['Date'][0:4])     ,axis=1)

In [374]:
#planecrash_df['Date'] = pd.to_datetime(planecrash_df['Date'])
#planecrash_df['Year'] = planecrash_df['Date'].dt.strftime('%Y')
#planecrash_df['Year'] = planecrash_df['Date'].astype(np.int32)
#planecrash_df

In [358]:
row['Year'] == 2019

True

In [355]:
(gdp_df["Year"]==row['Year'])

0        False
1        False
2        False
3        False
4        False
         ...  
15043    False
15044    False
15045    False
15046    False
15047    False
Name: Year, Length: 15048, dtype: bool

In [331]:
gdp_df[(gdp_df["Year"]=='2016') ]

,Country,Code,Year,GDP-Growth,GDP-Per-Capita,GDP
14784,Aruba,ABW,2016,NaN,NaN,NaN
14785,Afghanistan,AFG,2016,1.030660,563.825663,2.095875e+10
14786,Angola,AGO,2016,-2.133493,3229.619740,9.950382e+10
14787,Albania,ALB,2016,4.004413,5075.354985,1.454788e+10
14788,Andorra,AND,2016,1.629345,44569.783009,3.432141e+09
...,...,...,...,...,...,...
15043,Kosovo,XKX,2016,4.145372,4193.631327,7.738508e+09
15044,"Yemen, Rep.",YEM,2016,-2.701475,667.945437,1.903557e+10
15045,South Africa,ZAF,2016,0.787056,7439.919412,4.298757e+11
15046,Zambia,ZMB,2016,3.794901,1672.345428,2.901824e+10


In [277]:
gdp_df[(gdp_df["Country"]==row['Country'])  & (gdp_df["Year"]==row['Year']) ]

,Country,Code,Year,GDP-Growth,GDP-Per-Capita,GDP


In [375]:
for index, row in planecrash_df.iterrows():
    #pdb.set_trace()
    try:
        planecrash_df['GDP'] =gdp_df[(gdp_df["Country"]==row['Country'])  & (gdp_df["Year"]==row['Year']) ]['GDP'].values[0]
    except:
        planecrash_df['GDP'] = 0
        
planecrash_df[planecrash_df['GDP'].values != 0]

,Unnamed: 0,Unnamed: 0.1,Date,Time,Operator,Flight #,Route,AC Type,Registration,cn/ln,...,Ground,Summary,Location,Latitude,Longitude,geometry,Percent Survived,Country,Year,GDP


In [334]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/davidcoy/Downloads/My First Project-d821d4ac1a16.json"
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) 

/Users/davidcoy/Downloads/My First Project-d821d4ac1a16.json


In [335]:
import bq_helper
from bq_helper import BigQueryHelper #https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package

In [336]:
#https://www.kaggle.com/jessicali9530/how-to-query-world-development-indicators-data
wdi = bq_helper.BigQueryHelper(active_project="patents-public-data",
                                   dataset_name="worldbank_wdi")

In [337]:
# View table names under the worldbank_wdi data table
bq_assistant = BigQueryHelper("patents-public-data", "worldbank_wdi")
bq_assistant.list_tables()

['wdi_2016']

In [344]:
# View the first three rows of the wdi_2016 data table
df = bq_assistant.head("wdi_2016", num_rows=100)
df

,year,country_name,country_code,indicator_name,indicator_code,indicator_value
0,1960,Chad,TCD,Discrepancy in expenditure estimate of GDP (cu...,NY.GDP.DISC.CN,6.990439e+09
1,1961,Chad,TCD,"Death rate, crude (per 1,000 people)",SP.DYN.CDRT.IN,2.624000e+01
2,1961,Chad,TCD,GNI per capita (constant LCU),NY.GNP.PCAP.KN,2.846799e+05
3,1961,Chad,TCD,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R4.ZS,9.389671e-01
4,1961,Chad,TCD,"Merchandise imports by the reporting economy, ...",TM.VAL.MRCH.RS.ZS,9.243028e+01
...,...,...,...,...,...,...
95,1978,Chad,TCD,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,1.356000e+06
96,1979,Chad,TCD,Adjusted net national income per capita (curre...,NY.ADJ.NNTY.PC.CD,2.098689e+02
97,1979,Chad,TCD,Exports of goods and services (current US$),NE.EXP.GNFS.CD,1.785534e+08
98,1979,Chad,TCD,"External debt stocks, concessional (DOD, curre...",DT.DOD.ALLC.CD,1.767780e+08
